# 时间序列

时间序列(time series)数据都是一种重要的结构化数据形式。在**多个时间点**观察或测量到的**任何事物**都可以**形成一段时间序列**。**很多**时间序列是**固定频率**的,也就是说,数据点是根据**某种规律定期**出现的(比如每15秒、每5分钟、每月出现一次)。时间序列也**可以是不定期**的。时间序列数据的意义取决于具体的应用场景,主要有以下几种:
* 时间戳(timestamp),特定的时刻。
* 固定时期(period),如2007年1月或2010年全年。
* 时间间隔(interval),由起始和结束时间戳表示。时期(period)可以被看做间隔(interval)的特例。
* 实验或过程时间,每个时间点都是相对于特定起始时间的一个度量。例如,从放入烤箱时起,每秒钟饼干的直径。

主要讲解**前3种**时间序列。许多技术都可用于处理实验型时间序列,其索引可能是一个整数或浮点数(表示从实验开始算起已经过去的时间)。最简单也最常见的时间序列都是用时间戳进行索引的。pandas提供了一组**标准**的**时间序列处理工具**和**数据算法**。因此,你可以**高效处理**非常大的时间序列,轻松地进行**切片**/**切块**、**聚合**、对**定期**/**不定期**的时间序列进行**重采样**等。

In [42]:
# -*- coding: utf-8 -*-

from datetime import datetime,timedelta
from dateutil.parser import parse

from pandas import Series,DataFrame
import pandas as pd

import numpy as np

## 日期和时间数据类型及工具

### python内置datetime

#### 基本使用

In [2]:
datetime.now()

datetime.datetime(2018, 4, 18, 14, 7, 48, 140892)

In [4]:
datetime.now() - timedelta(2) # 2代表的是两天

datetime.datetime(2018, 4, 16, 14, 8, 37, 905896)

#### datetime模块的数据类型

* date：以公历形式存储日历信息（年、月、日）；
* time：将时间存储为时、分、秒、毫秒；
* datetime：存储日期和时间；
* timedelta：表示两个datetime之间的时间差（日，秒，毫秒）；

### 字符串和datetime相互转换

#### 日期转字符串

##### str

In [8]:
str(datetime.now())

'2018-04-18 14:14:45.638722'

##### datetime.strftime -- 定制输出格式

In [24]:
datetime.now().strftime('%Y - %m - %d %H : %m : %S')

'2018 - 04 - 18 14 : 04 : 02'

#### 字符串转日期

##### datetime.strptime

In [30]:
datetime.strptime('2012-01-01', '%Y-%m-%d')

datetime.datetime(2012, 1, 1, 0, 0)

#### dateutil解析常用日期格式

##### 解析常见格式

In [32]:
parse('2012-01-02')

datetime.datetime(2012, 1, 2, 0, 0)

In [33]:
parse('2012 01 02 15:23:23')

datetime.datetime(2012, 1, 2, 15, 23, 23)

In [36]:
parse('2012/03/03')

datetime.datetime(2012, 3, 3, 0, 0)

##### 解析日在月前面的情况

In [38]:
parse('2012/28/11', dayfirst=True)

datetime.datetime(2012, 11, 28, 0, 0)

#### pandas.to_datetime -- 特有的NaT是Not a time的意思，时间类型的NA值

In [44]:
pd.to_datetime(['2018/02/02','2000-12-11',None])

DatetimeIndex(['2018-02-02', '2000-12-11', 'NaT'], dtype='datetime64[ns]', freq=None)

#### datetime格式定义

基本格式定义：
* %Y：4位数的年 -- 2012；
* %y：2位数的年 -- 12代表2012；
* %m：2位数的月 -- 02代表2月（1~12）；
* %d：2位数的日 -- 1~31；
* %H：时 -- 00~23（24小时制）；
* %I：时 -- 01~12（12小时制）；
* %M：2位数的分（00~59）；
* %S：秒（00~61）；
* %w：整数表示星期几（0（星期天）~6）；
* %U：每年的第几周（00~53），星期天为一周的第一天，每年第一个星期天之前的那几天统称为第0周；
* %W：每年的第几周（00~53），星期一为一周的第一天，每年第一个星期一之前的那几天统称为第0周；
* %z：以+HHMM或-HHMM表示的UTC时区偏移量，如果时区为native，则返回空字符串；
* %F：%Y-%m-%d，例如2012-01-01；
* %D：%m/%d/%y，例如04/18/12；

针对特定于当前环境：
* %a：星期几的简写；
* %A：星期几的全称；
* %b：月份的简写；
* %B：月份的全称；
* %c：完整的日期和时间，例如“Tue 01 May 2012 04:20:20 pm”；
* %p：不同环境中的AM和PM；
* %x：适用于当前环境的日期格式，例如，在美国，“May 1,2012”会产生“05/01/2012”；
* %X：适用于当前环境的时间格式，例如“04:24:12 pm”；

## 时间序列基础